# 模型的保存与加载

模型训练好需要保存，以备后续使用。

主要分为三块：
- 序列化与反序列化
- 模型保存与加载
- 断点的续训练技术

## 序列化与反序列化

**序列化**：内存中某一个对象保存到硬盘中，以二进制序列的形式存储下来。

**反序列化**：将硬盘中存储的二进制数，反序列化到内存中，得到一个相应的对象，这样就可以再次使用这个模型。

<img style="float: center;" src="images/207.png" width="70%">

序列化与反序列的目的就是将模型长久地保存。

PyTorch中序列化与反序列化地方法：
- torch.save(obj, f)：obj表示对象，即保存地数据，可以是模型，张量，dict等，f表示输出的路径。
- torch.load(f, map_location)：f表示文件路径，map_location：指定存放位置（CPU或GPU）

## 模型保存与加载

PyTorch模型保存有两种方法：一种是保存整个Module，另一种是保存模型的参数。
- 保存和加载整个Module（比较费时且占内存）：
  - torch.save(net, path)
  - torch.load(fpath)
- 保存模型参数（只保留模型上可学习的部分）：
  - torch.save(net.state_dict(), path)
  - net.load_state_dict(torch.load(path))

In [1]:
# 先建立一个网络模型
import torch.nn as nn

class LeNet2(nn.Module):
    def __init__(self, classes):
        super(LeNet2, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            
            nn.Linear(120, 84),
            nn.ReLU(),
            
            nn.Linear(84, classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x

    def initialize(self):
        for p in self.parameters():
            p.data.fill_(20191104)
     

# 建立一个网络
net = LeNet2(classes=2019)

# 训练
print("训练前: ", net.features[0].weight[0, ...])
net.initialize()
print("训练后: ", net.features[0].weight[0, ...])

训练前:  tensor([[[ 0.0837, -0.0572, -0.1029,  0.0314,  0.0044],
         [-0.1025, -0.0942,  0.0581, -0.0094, -0.0580],
         [ 0.0902, -0.0722, -0.0730,  0.1056, -0.0366],
         [ 0.0383,  0.0003,  0.0471, -0.0025,  0.0540],
         [ 0.0701,  0.0658,  0.0750,  0.0424,  0.1124]],

        [[-0.0773, -0.1056,  0.0173,  0.0423, -0.0592],
         [-0.0521, -0.0434, -0.0356,  0.0811, -0.0699],
         [-0.0042, -0.0912,  0.0293,  0.0255, -0.0623],
         [-0.0315, -0.0111, -0.0591,  0.0079, -0.0946],
         [ 0.0750, -0.0129, -0.0562,  0.1045,  0.0169]],

        [[ 0.0971,  0.1123,  0.0168, -0.0685,  0.0672],
         [-0.0023, -0.1074,  0.0327,  0.0992,  0.0624],
         [-0.0216,  0.1024,  0.0504, -0.0641,  0.0715],
         [ 0.1151, -0.0461, -0.0337,  0.1093, -0.0373],
         [ 0.0319,  0.0424,  0.0753, -0.0246,  0.0598]]],
       grad_fn=<SelectBackward0>)
训练后:  tensor([[[20191104., 20191104., 20191104., 20191104., 20191104.],
         [20191104., 20191104., 20191104.,

In [2]:
import torch

path_model = "./model.pkl"
path_state_dict = "./model_state_dict.pkl"

# 保存整个模型
torch.save(net, path_model)

# 保存模型参数
torch.save(net.state_dict(), path_state_dict)

下面保存整个模型和保存模型参数：
<img style="float: center;" src="images/208.png" width="70%">

In [3]:
# 如果保存的是整个模型，导入比较简单
net_load = torch.load(path_model)
net_load

LeNet2(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=2019, bias=True)
  )
)

In [4]:
# 只保留参数的导入
# 首先需要把模型的参数加载进来
state_dict_load = torch.load(path_state_dict)
print(state_dict_load)

# 之后需要新建一个网络
net_new = LeNet2(classes=2019)

print("加载前：", net_new.features[0].weight[0, ...])
# 把保存的模型参数加入到网络中
net_new.load_state_dict(state_dict_load)
print("加载后：", net_new.features[0].weight[0, ...])

OrderedDict([('features.0.weight', tensor([[[[20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.]],

         [[20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.]],

         [[20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 20191104., 20191104., 20191104., 20191104.],
          [20191104., 2

## 模型的断点续训练

当模型训练的时间非常长时，而训练到了中途出现一些意外情况（断电，死机等），当再次来电时，肯定希望模型在中途训练的地方继续往下训练，这就需要在模型训练的过程中保存一些断点。

当意外发生时，模型可以从断点处继续训练而不是从头开始。

断点（checkpoint）需要保留哪些参数？
- 模型训练5个步骤：数据->模型->损失函数->优化器->迭代训练
- 5个步骤中，数据，损失函数无法改变，迭代训练过程中，模型里的可学习参数，优化器里的一些缓存是会变的，因此需要保留这些东西。

因此断点中需要保存模型的数据，优化器的数据，还有迭代到了第几次：
```python
checkpoint={
    "model_state_dict":net.state_dict(),
    "optimizer_state_dict":optimizer.state_dict(),
    "epoch":epoch
}
```

通过人民币二分类实验，模拟一个训练过程中的意外中断和恢复，看看怎么使用断点续训练：
<img style="float: center;" src="images/209.png" width="70%">

上面发生了一个意外中断，即我们设置了断点并且进行保存，则下面进行恢复，从断点处进行训练，即第6个epoch开始，恢复断点训练：
<img style="float: center;" src="images/210.png" width="70%">

在模型训练过程中，以一定的间隔去保存模型，保存断点，在断点里不仅要保存模型的参数，还要保存优化器的参数以及迭代的次数，这样才可以在意外中断之后恢复训练。

# 模型的微调

迁移学习：机器学习的分支，研究源域的只是如何应用到目标域，将源任务中学习到的知识运用到目标任务当中，用于提升目标任务里模型的性能。
<img style="float: center;" src="images/211.png" width="70%">

当某个任务的数据比较少的时候，没法训练一个好的模型时，可以采用迁移学习的思路，把**类似任务**训练好的模型给迁移过来。

由于这种模型已经在原来的任务上训练的差不多了，迁移到新任务上之后，只需要微调一些参数，往往就能比较好的应用于新的任务。

当然，需要在原来模型的基础上修改输出部分，毕竟任务不同，输出可能不同。

注意：一定要在类似任务上的模型迁移（不要试图将一个NLP模型迁移到CV里）
<img style="float: center;" src="images/212.png" width="70%">

模型微调步骤：
1. 获取预训练模型参数（源任务中学习到的知识）
2. 加载模型（load_state_dict），将学习到的知识放到新的模型
3. 修改输出层，以适应新的任务

模型微调训练方法：
- 固定预训练的参数（requires_grad=False, lr = 0）
- Features Extractor较小学习率（params_group）

## 示例

用训练好的ResNet-18进行二分类：让模型分出蚂蚁和蜜蜂
<img style="float: center;" src="images/213.png" width="70%">

训练集120张，验证集70张（可以看出数据比较少），如果新建模型进行训练预测，效果较差。

因此考虑采用训练好的ResNet-18来完成任务。

首先看ResNet-18的结构，看看哪里可以改动：
<img style="float: center;" src="images/214.png" width="70%">

具体使用：
<img style="float: center;" src="images/215.png" width="70%">

当然，也可以不冻结前面的层，这样的话需要修改前面参数学习率。因为我们的优化器里有参数组的概念，可以把网络的前面和后面分成不同的参数组，使用不同的学习率进行训练，当前面的学习率为0的时候，与冻结前面的层是一样的效果，但是这种写法比较灵活。
<img style="float: center;" src="images/216.png" width="70%">

通过模型的迁移，可以发现这个任务会完成地比较好。

# GPU

## CPU VS GPU

CPU（Central Processing Unit，中央处理器）：控制器+运算器

GPU（Graphics Processing Unit，图形处理器）：处理统一的，无依赖地大规模数据运算
<img style="float: center;" src="images/217.png" width="70%">

## 数据迁移至GPU

PyTorch数据主要有两种：Tensor和Module
- CPU->GPU：data.to('cuda')
- GPU->CPU：data.to('cpu')

### to函数

转换数据类型/设备

In [5]:
# tensor.to(*args, **kwargs)
x = torch.ones((3, 3))
x = x.to(torch.float64)  # 转换数据类型

x = torch.ones((3, 3))
x = x.to('cuda')  # 设备转移

In [6]:
# module.to(*args, **kwargs)
linear = nn.Linear(2, 2)
linear.to(torch.double)  # 模型里可学习参数地数据类型变成float64

gpu1 = torch.device('cuda')
linear.to(gpu1)  # 模型从CPU迁移到GPU

Linear(in_features=2, out_features=2, bias=True)

注意：
- 张量不执行inplace，因此需要等号重新赋值
- 模型执行inplace，因此不需要等号重新赋值

张量：
<img style="float: center;" src="images/218.png" width="70%">

模型：
<img style="float: center;" src="images/219.png" width="70%">

如果模型在GPU上，则数据也必须在GPU上才能运行。

**数据与模型必须在相同地设备上。**

### torch.cuda常用方法

- torch.cuda.device_count()：计算当前可见可用的GPU数
- torch.cuda.get_device_name()：获取GPU名称
- torch.cuda.manual_seed()：为当前GPU设置随机种子
- torch.cuda.manual_seed_all()：为所有可见可用GPU设置随机种子

- torch.cuda.set_device()：设置主GPU（默认GPU）为哪一个物理GPU（不推荐）
  - 推荐的方式是设置系统的环境变量：os.environ.setdefault("CUDA_VISIBLE_DEVICES", "2,3")
  - 通过该方式合理分配GPU，可以避免多人使用的时候冲突
  - 注意，这里出2，3指的是物理GPU的2，3，但在逻辑GPU上，表示0，1
  <img style="float: center;" src="images/220.png" width="70%">
  - 如果设置0，3，2，则对应顺序如下：
  <img style="float: center;" src="images/221.png" width="70%">
  
在逻辑GPU中，有一个主GPU的概念（通常指GPU0），常用于多GPU并行计算中。

## 多GPU并行运算

有若干块GPU（例如4块），有一个主GPU，当拿到样本数据（例如主GPU拿到16个样本），它会经过16/4=4的运算，把数据分成4份，自己留一份，其余3份分发到另外3块GPU上进行计算，等其他GPU计算结束后，主GPU再把结果收回来负责整合。

多GPU并行分为三步：分发->并行计算->收回结果整合

PyTorch中多GPU并行运算机制：

torch.nn.DataParallel：包装模型，实现分发并行机制

torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)

- module：需要包装分发的模型
- device_idds：可分发的gpu，默认分发到所有可见可用GPU，通常这个参数不用管，在环境变量中设置
- output_device：结果输出设备，通常输出到主GPU

<img style="float: center;" src="images/222.png" width="70%">

<img style="float: center;" src="images/223.png" width="70%">

In [7]:
# 多GPU，查看每一块GPU缓存进行排序，结果作为逻辑GPU使用
# 排在最前面的是主GPU
import numpy as np
import os
import platform

def get_gpu_memory():    
    if 'Windows' == platform.system():
        os.system('nvidia-smi -q -d Memory | grep -A4 GPU | grep Free > tmp.txt')
        memory_gpu = [int(x.split()[2]) for x in open('tmp.txt', 'r').readlines()]
        os.system('rm tmp.txt')
    else:
        memory_gpu = False
        print("显存计算功能暂不支持windows操作系统")
    return memory_gpu


gpu_memory = get_gpu_memory()
print(gpu_memory)

if not gpu_memory:
    print("\ngpu free memory: {}".format(gpu_memory))

gpu_list = np.argsort(gpu_memory)[::-1]
gpu_list_str = ','.join(map(str, gpu_list))
os.environ.setdefault("CUDA_VISIBLE_DEVICES", gpu_list_str)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[]

gpu free memory: []


## GPU模型加载中常见的两个问题

<img style="float: center;" src="images/224.png" width="70%">
报错原因是模型是以cuda形式进行保存（即在GPU上训练完保存），保存完之后想在一个没有GPU的机器上使用这个模型，就会报上面的错误。

解决办法：
- torch.load(path_state_dict, map_location="cpu")
- 这样就可以在CPU设备上加载GPU上保存的模型了

<img style="float: center;" src="images/225.png" width="70%">

报错信息出现在多GPU并行运算的机制训练好了某个模型并保存，然后想建立一个普通的模型使用保存好的这些参数，就会报这个错误。

原因：在多GPU并行运算时，模型net先进行一个并行的包装，这个包装使得每一层得参数名称前面会加一个module，这是如果想把这些参数转移到普通得net中，发现找不到这种module.开头的参数（即无法匹配），因为普通的net里面的参数时没有前面的module的。

这个时候需要重新创建一个字典，把名字改了之后再导入。

首先先在多GPU的环境下，建立一个网络，并且进行包装，放到多GPU环境上训练保存：
<img style="float: center;" src="images/226.png" width="70%">

加载的时候报错：
<img style="float: center;" src="images/227.png" width="70%">

解决办法：
```python
from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in state_dict_load.items():
    namekey = k[7:] if k.startswith('module.') else k
    new_state_dict[namekey] = v
    
print("new_state_dict:\n{}".format(new_state_dict))

net.load_state_dict(new_state_dict)

```

效果：
<img style="float: center;" src="images/228.png" width="70%">